In [1]:
data = open('HP1.txt','r', encoding="utf8").read();
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(data_size,", ",vocab_size)

char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(char_to_ix)
print(ix_to_char)

def encode(idx,num_entry):
    ret = np.zeros(num_entry)
    ret[idx] = 1
    return ret;

def encode_array(array,num_entry):
    xs = np.zeros((len(array),num_entry))
    for i in range(len(array)):
        xs[i][array[i]] = 1; 
    return xs;

431677 ,  79
{'E': 0, 'u': 1, 'L': 2, 'i': 65, 'l': 5, 'Z': 6, '9': 7, '3': 55, '7': 27, 'A': 8, '(': 10, ' ': 11, 'O': 25, 'D': 12, 'd': 13, '?': 16, 'B': 15, 'U': 18, 'a': 17, ';': 3, 'K': 66, 'w': 20, 'P': 21, 'M': 22, 'v': 23, '2': 29, 'f': 26, 'e': 28, '\n': 30, '\\': 33, 'Y': 34, '4': 35, '!': 38, 'm': 37, 'q': 19, '~': 41, '-': 74, 'j': 42, "'": 43, 'z': 49, 'p': 59, '5': 44, 'S': 45, 't': 47, 'J': 50, 'W': 73, 'x': 51, '\t': 52, 'I': 53, 'C': 14, 'R': 36, 'G': 56, 'h': 57, 's': 60, '*': 61, ':': 39, '1': 62, 'Q': 75, 'T': 4, 'V': 63, '"': 48, '8': 69, 'b': 76, 'c': 64, '.': 40, '0': 67, 'N': 77, 'g': 68, 'n': 78, 'H': 70, 'k': 71, 'r': 72, 'F': 31, '6': 9, 'o': 58, 'X': 32, 'y': 24, ',': 46, ')': 54}
{0: 'E', 1: 'u', 2: 'L', 3: ';', 4: 'T', 5: 'l', 6: 'Z', 7: '9', 8: 'A', 9: '6', 10: '(', 11: ' ', 12: 'D', 13: 'd', 14: 'C', 15: 'B', 16: '?', 17: 'a', 18: 'U', 19: 'q', 20: 'w', 21: 'P', 22: 'M', 23: 'v', 24: 'y', 25: 'O', 26: 'f', 27: '7', 28: 'e', 29: '2', 30: '\n', 31: 'F', 32

In [55]:
import numpy as np
import copy

# Helper functions
def softmax(array):
    return np.exp(array)/ np.sum(np.exp(array)) # return an array

def sigmoid(x):
    return (1/(1+np.exp(-x)))

def sigmoid_deriv(y):
    return (y*(1-y))

def tanh(x):
    return np.tanh(x)

def tanh_deriv(y):
    return 1 - pow(np.tanh(y),2)

# RNN
class myRNN:
    
    def __init__ (self, lenIn, lenOut, lenRec, sizeHidden, inputs_encoded, targets, learningRate):
        
        # Hyper parameters
        self.lenIn          = lenIn
        self.lenOut         = lenOut
        self.lenRec         = lenRec
        self.sizeHidden     = sizeHidden
        self.learningRate   = learningRate
        
        # input & expected output
        self.inputs_encoded = inputs_encoded;
        self.targets = targets;
        
        # parameters for inference
        self.x  = np.zeros(lenIn)  
        self.y  = np.zeros(lenOut)
        self.h  = np.zeros((2,sizeHidden))
        self.c  = np.zeros((2,sizeHidden))
        
        self.W  = np.zeros((lenOut,sizeHidden)) # for the last fully connected layer
        self.GW = np.zeros((lenOut,sizeHidden)) # Gradient, for W-update using RMSprop
        self.b  = np.zeros(lenOut)
        self.Gb = np.zeros(lenOut)
        
        # for training phase 
        self.xs = np.zeros((lenRec,lenIn))
        self.ys = np.zeros((lenRec,lenOut))
        self.cs = np.zeros((2,lenRec,sizeHidden))
        self.hs = np.zeros((2,lenRec,sizeHidden))
        
        # for training phase bookkeeping
        self.fg = np.zeros((2,lenRec,sizeHidden)) # forget gate
        self.ig = np.zeros((2,lenRec,sizeHidden)) # input  gate
        self.og = np.zeros((2,lenRec,sizeHidden)) # output gate
        self.mc = np.zeros((2,lenRec,sizeHidden)) # memory cell state (candidate)
        
        # LSTM class
        self.LSTM0 = LSTM(sizeHidden+lenIn,sizeHidden,lenRec,learningRate)
        self.LSTM1 = LSTM(sizeHidden*2    ,sizeHidden,lenRec,learningRate)
        ''' end of myRNN.__init__ '''
       
    ''' This is used when mini-batch is used '''            
    def update_inputs_targets(self, inputs_encoded, targets):
        self.inputs_encoded  = inputs_encoded
        self.targets         = targets
    
    def fwd_pass(self): 
        prev_h0 = np.zeros_like(self.hs[0][0])
        prev_h1 = np.zeros_like(self.hs[0][0])
        
        for t in range(0,self.lenRec):
            # update input
            self.x    = self.inputs_encoded[t]
            self.xs[t]= self.inputs_encoded[t]
            
            self.LSTM0.hx = np.hstack((prev_h0, self.x));         
            c, h, f, i, m, o = self.LSTM0.fwd_pass()
            # bookkeeping
            self.cs[0][t] = c
            self.hs[0][t] = h
            self.fg[0][t] = f
            self.ig[0][t] = i
            self.mc[0][t] = m
            self.og[0][t] = o
            prev_h0 = self.hs[0][t]
            
            self.LSTM1.hx = np.hstack((prev_h1, h));      
            #self.LSTM1.hx = np.hstack((prev_h1, self.x)); 
            c, h, f, i, m, o = self.LSTM1.fwd_pass()
            # bookkeeping
            self.cs[1][t] = c
            self.hs[1][t] = h
            self.fg[1][t] = f
            self.ig[1][t] = i
            self.mc[1][t] = m
            self.og[1][t] = o            
            prev_h1 = self.hs[1][t]
            
            # output layer - fully connected layer
            self.ys[t] = np.dot(self.W,self.hs[1][t]) + self.b      
            
        return;              
    
    def bwd_pass(self):        

        avg_loss = 0; # using cross entropy average
        c2next_grad  = np.zeros((2,self.sizeHidden))
        h2next_grad  = np.zeros((2,self.sizeHidden))
        
        # output bp
        W_grad   = np.zeros((self.lenOut,self.sizeHidden))
        b_grad  = np.zeros(self.lenOut)
        
        # LSTM internal bp
        hxf1_grad   = np.zeros((self.sizeHidden,self.LSTM1.lenIn));
        hxi1_grad   = np.zeros((self.sizeHidden,self.LSTM1.lenIn));
        hxm1_grad   = np.zeros((self.sizeHidden,self.LSTM1.lenIn));
        hxo1_grad   = np.zeros((self.sizeHidden,self.LSTM1.lenIn));
        
        hxf0_grad   = np.zeros((self.sizeHidden,self.LSTM0.lenIn));
        hxi0_grad   = np.zeros((self.sizeHidden,self.LSTM0.lenIn));
        hxm0_grad   = np.zeros((self.sizeHidden,self.LSTM0.lenIn));
        hxo0_grad   = np.zeros((self.sizeHidden,self.LSTM0.lenIn));
        
        fb_grad   = np.zeros((2,self.sizeHidden))
        ib_grad   = np.zeros((2,self.sizeHidden))
        mb_grad   = np.zeros((2,self.sizeHidden))
        ob_grad   = np.zeros((2,self.sizeHidden))
                   
        # propagates through time and layers

        for t in reversed(range(0,self.lenRec)):
            
            prob = softmax(self.ys[t]) # prevent zero
            prob_fix  = prob + 1e-9
                
            # cross entropy
            err       = np.log(prob_fix[int(self.targets[t])])
            avg_loss += err
     
            dy = copy.deepcopy(prob)
            dy[int(self.targets[t])] -= 1
            W_grad += np.dot((np.atleast_2d(dy)).T,np.atleast_2d(self.hs[1][t]))
            b_grad += dy
            
            dh = np.dot(self.W.T,dy) + h2next_grad[1]
            # Second Layer
            
            if(t > 0):
                prev_h1 = self.hs[1][t-1]
                prev_c1 = self.cs[1][t-1]
            else:
                prev_h1 = np.zeros_like(self.hs[1][0])
                prev_c1 = np.zeros_like(self.cs[1][0])
            ################################################
            self.LSTM1.hx = np.hstack((prev_h1,self.hs[0][t]));
            self.LSTM1.c = self.cs[1][t];

            # LSTM bwd
            dhxf,dhxi,dhxm,dhxo, dbf,dbi,dbm,dbo,\
            c2next_grad[1],h2next_grad[1],x_grad = \
            self.LSTM1.bwd_pass( dh, prev_c1 ,self.fg[1][t],self.ig[1][t],\
                               self.mc[1][t],self.og[1][t],\
                                 c2next_grad[1]);
            
            hxf1_grad   +=  dhxf;
            hxi1_grad   +=  dhxi;  
            hxm1_grad   +=  dhxm;  
            hxo1_grad   +=  dhxo;   
            fb_grad[1]  +=  dbf;
            ib_grad[1]  +=  dbi;
            mb_grad[1]  +=  dbm;
            ob_grad[1]  +=  dbo;

            # First layer
            ###########################
            
            dh = x_grad + h2next_grad[0];
            
            if(t > 0):
                prev_h0 = self.hs[0][t-1]
                prev_c0 = self.cs[0][t-1]
            else:
                prev_h0 = np.zeros_like(self.hs[0][0])
                prev_c0 = np.zeros_like(self.cs[0][0])
                
            self.LSTM0.hx = np.hstack((prev_h0,self.xs[t]));
            self.LSTM0.c = self.cs[0][t];
            
            # LSTM bwd
            dhxf,dhxi,dhxm,dhxo, dbf,dbi,dbm,dbo, \
            c2next_grad[0],h2next_grad[0],x_grad = \
            self.LSTM0.bwd_pass( dh, prev_c0 ,self.fg[0][t],self.ig[0][t],\
                               self.mc[0][t],self.og[0][t],\
                                 c2next_grad[0]);
            
            hxf0_grad +=  dhxf;
            hxi0_grad +=  dhxi;  
            hxm0_grad +=  dhxm;  
            hxo0_grad +=  dhxo;   
            fb_grad[0]  +=  dbf;
            ib_grad[0]  +=  dbi;
            mb_grad[0]  +=  dbm;
            ob_grad[0]  +=  dbo;
            
        # update using RMSprop
        self.LSTM1.update(hxf1_grad/self.lenRec, hxi1_grad/self.lenRec, \
                           hxm1_grad/self.lenRec, hxo1_grad/self.lenRec, \
                           fb_grad[1]/self.lenRec, ib_grad[1]/self.lenRec, \
                           mb_grad[1]/self.lenRec, ob_grad[1]/self.lenRec);

        self.LSTM0.update(hxf0_grad/self.lenRec, hxi0_grad/self.lenRec, \
                           hxm0_grad/self.lenRec, hxo0_grad/self.lenRec, \
                           fb_grad[0]/self.lenRec, ib_grad[0]/self.lenRec, \
                           mb_grad[0]/self.lenRec, ob_grad[0]/self.lenRec);
        
        self.update(W_grad/self.lenRec,b_grad/self.lenRec);
        
        return avg_loss/self.lenRec;
            
          
            
    def update(self, W_grad, b_grad):
        self.GW = 0.9*self.GW + 0.1*W_grad**2;
        self.W -= self.learningRate/np.sqrt(self.GW + 1e-8) * W_grad;
        self.Gb = 0.9*self.Gb + 0.1*b_grad**2;
        self.b -= self.learningRate/np.sqrt(self.Gb + 1e-8) * b_grad;

    def inference(self,x):
        # update input
        self.x = x
        self.LSTM0.hx = np.hstack((self.h[0],self.x))
        c0, h0, f0, i0, m0, o0 = self.LSTM0.fwd_pass()
        self.c[0] = c0
        self.h[0] = h0

        self.LSTM1.hx = np.hstack((self.h[1],self.h[0]))
        c1, h1, f1, i1, m1, o1 = self.LSTM1.fwd_pass()
        self.c[1] = c1
        self.h[1] = h1
        
        # output layer - may replace with softmax instead
        self.y = np.dot(self.W,self.h[1]) + self.b
        p   = softmax(self.y)     
        return np.random.choice(range(self.lenOut), p=p.ravel())
  


In [56]:
class LSTM:
    
    def __init__ (self,lenIn,sizeHidden,lenRec,learningRate):
        self.lenIn        = lenIn
        self.sizeHidden   = sizeHidden
        self.lenRec       = lenRec
        self.learningRate = learningRate
        
        # hx == x is x and h horizontally stacked together
        self.hx = np.zeros(lenIn)
        self.c = np.zeros(sizeHidden)
        self.h = np.zeros(sizeHidden)
        
        # Weight matrices
        self.fW = np.random.random((sizeHidden,lenIn));
        self.iW = np.random.random((sizeHidden,lenIn));
        self.mW = np.random.random((sizeHidden,lenIn)); # cell state
        self.oW = np.random.random((sizeHidden,lenIn));
                             
        # biases
        self.fb = np.zeros(sizeHidden);
        self.ib = np.zeros(sizeHidden); 
        self.mb = np.zeros(sizeHidden); 
        self.ob = np.zeros(sizeHidden); 
               
        # for RMSprop only
        self.GfW = np.random.random((sizeHidden,lenIn));
        self.GiW = np.random.random((sizeHidden,lenIn));
        self.GmW = np.random.random((sizeHidden,lenIn)); 
        self.GoW = np.random.random((sizeHidden,lenIn));
                             
        self.Gfb = np.zeros(sizeHidden);
        self.Gib = np.zeros(sizeHidden); 
        self.Gmb = np.zeros(sizeHidden);
        self.Gob = np.zeros(sizeHidden); 
        
        ''' end of LSTM.__init__ '''
        
    def fwd_pass(self):
        f       = sigmoid(np.dot(self.fW, self.hx) + self.fb)
        i       = sigmoid(np.dot(self.iW, self.hx) + self.ib)
        m       = tanh(   np.dot(self.mW, self.hx) + self.mb)        
        o       = sigmoid(np.dot(self.oW, self.hx) + self.ob)
        self.c *= f
        self.c += i * m
        self.h  = o * tanh(self.c)
        
        return self.c, self.h, f, i, m, o;
    
    def bwd_pass(self, dh, prev_c, f, i, m, o, c_g):
        
        dh = np.clip(dh, -6, 6);       
        # h = o*tanh(c)
        do  = tanh(self.c) * dh
        do  = sigmoid_deriv(o)*do
        dhxo = np.dot((np.atleast_2d(do)).T,np.atleast_2d(self.hx)) 
        
        # h = o*tanh(c) - add c_g (c_grad in next timestep, account for the branch here)
        dcs = dh * o * tanh_deriv(self.c) + c_g
        dcs = np.clip(dcs, -6, 6); 
        
        # c = c_prev * f + m * i
        dm = i * dcs
        dm = tanh_deriv(m) * dm
        dhxm = np.dot((np.atleast_2d(dm)).T,np.atleast_2d(self.hx)) 
        
        # c = c_prev * f + m * i
        di  = m * dcs
        di  = sigmoid_deriv(i) * di
        dhxi = np.dot((np.atleast_2d(di)).T,np.atleast_2d(self.hx)) 
        
        # c = c_prev * f + m * i
        df = prev_c * dcs
        df = sigmoid_deriv(f) * df
        dhxf = np.dot((np.atleast_2d(df)).T,np.atleast_2d(self.hx)) 
        
        # c = c_prev * f + m * i
        c_grad  = dcs * f
        hx_grad = np.dot(self.fW.T, df) + np.dot(self.iW.T, di) +\
                          np.dot(self.oW.T, do) + np.dot(self.mW.T, dm)
        
        
        return dhxf,dhxi,dhxm,dhxo,df,di,dm,do,\
    c_grad,hx_grad[:self.sizeHidden],hx_grad[self.sizeHidden:];
    
    def update(self, f_grad, i_grad, m_grad, o_grad, fb_grad, ib_grad, mb_grad, ob_grad):

        self.GfW = 0.9*self.GfW + 0.1*f_grad**2
        self.GiW = 0.9*self.GiW + 0.1*i_grad**2
        self.GmW = 0.9*self.GmW + 0.1*m_grad**2
        self.GoW = 0.9*self.GoW + 0.1*o_grad**2
        
        self.Gfb = 0.9*self.Gfb + 0.1*fb_grad**2
        self.Gib = 0.9*self.Gib + 0.1*ib_grad**2
        self.Gmb = 0.9*self.Gmb + 0.1*mb_grad**2
        self.Gob = 0.9*self.Gob + 0.1*ob_grad**2
        
        self.fW -= self.learningRate/np.sqrt(self.GfW + 1e-8) * f_grad
        self.iW -= self.learningRate/np.sqrt(self.GiW + 1e-8) * i_grad
        self.mW -= self.learningRate/np.sqrt(self.GmW + 1e-8) * m_grad
        self.oW -= self.learningRate/np.sqrt(self.GoW + 1e-8) * o_grad
        self.fb -= self.learningRate/np.sqrt(self.Gfb + 1e-8) * fb_grad
        self.ib -= self.learningRate/np.sqrt(self.Gib + 1e-8) * ib_grad
        self.mb -= self.learningRate/np.sqrt(self.Gmb + 1e-8) * mb_grad
        self.ob -= self.learningRate/np.sqrt(self.Gob + 1e-8) * ob_grad
        

In [57]:
seq_length,position = 75,0
inputs = [char_to_ix[ch] for ch in data[position:position+seq_length]]
print(data[position:position+seq_length])
print("inputs",inputs)

targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 
print(data[position+1:position+seq_length+1])
print("targets",targets)

n,position = 0,0;
epoch = 20*1000;
lenIn, lenOut, lenRec = vocab_size,vocab_size, seq_length
sizeHidden, numHiddenLayer = 64,1;
learningRate = 0.1;


R = myRNN(lenIn, lenOut, lenRec, sizeHidden, encode_array(inputs,vocab_size),targets, learningRate)

# training
while n<epoch:
    
    if(position+seq_length+1 >= len(data) or n == 0):
        print("!!!!",len(data))
        position = 0;
        
    inputs  = [char_to_ix[ch] for ch in data[position:position+seq_length]]
    targets = [char_to_ix[ch] for ch in data[position+1:position+seq_length+1]] 

    R.update_inputs_targets(encode_array(inputs,vocab_size),targets)
    R.fwd_pass();
    
    err = R.bwd_pass();
    
    if(n%500 == 0):
        print(n,"err:",err)
        seed = encode(n % vocab_size,vocab_size)
        print(ix_to_char[n % vocab_size])
        result = [];
        R.h  = np.zeros_like(R.h)
        R.c  = np.zeros_like(R.c)
        for i in range(40):
            ret = R.inference(seed)
            #print(i,":",ret)
            result.append(ret)
            seed = encode(ret,vocab_size)
        decode = ''.join([ix_to_char[ch] for ch in result] )
        print(decode)

    position += seq_length;
    n += 1;

Harry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and
inputs [70, 17, 72, 72, 24, 11, 21, 58, 47, 47, 28, 72, 11, 17, 78, 13, 11, 47, 57, 28, 11, 45, 58, 72, 64, 28, 72, 28, 72, 43, 60, 11, 45, 47, 58, 78, 28, 30, 14, 70, 8, 21, 4, 0, 36, 11, 25, 77, 0, 30, 4, 70, 0, 11, 15, 25, 34, 11, 73, 70, 25, 11, 2, 53, 63, 0, 12, 30, 22, 72, 40, 11, 17, 78, 13]
arry Potter and the Sorcerer's Stone
CHAPTER ONE
THE BOY WHO LIVED
Mr. and 
targets [17, 72, 72, 24, 11, 21, 58, 47, 47, 28, 72, 11, 17, 78, 13, 11, 47, 57, 28, 11, 45, 58, 72, 64, 28, 72, 28, 72, 43, 60, 11, 45, 47, 58, 78, 28, 30, 14, 70, 8, 21, 4, 0, 36, 11, 25, 77, 0, 30, 4, 70, 0, 11, 15, 25, 34, 11, 73, 70, 25, 11, 2, 53, 63, 0, 12, 30, 22, 72, 40, 11, 17, 78, 13, 11]
!!!! 431677
0 err: -4.36944777346702
E
rdYdPn aSOYrSOeROPSsLraesnOHe
STSthPryae
500 err: -8.20991230627229
f
DDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD


KeyboardInterrupt: 